In [2]:
import pandas as pd
import numpy as np
from model import get_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import ReLU

In [3]:
rain = pd.read_csv("./Data/Rainfall.csv")

In [15]:
# 1948-2000
slp_s,  slp_e  = 0,  53
uwnd_s, uwnd_e = 0,  53
sst_s,  sst_e  = 94, 147
rain_s, rain_e = 47, 99

In [18]:
import glob

slp_path  = "./Data/slp/*"
sst_path  = "./Data/sst/*"
uwnd_path = "./Data/uwnd/*"

slp_months = []
for i in glob.glob(slp_path):
  slp_months.append(i)

sst_months = []
for i in glob.glob(sst_path):
  sst_months.append(i)

uwnd_months = []
for i in glob.glob(uwnd_path):
  uwnd_months.append(i)
  
slp_months.sort()
sst_months.sort()
uwnd_months.sort()

In [19]:
# Data form 1948-2000
import statistics
lpa = statistics.mean(list(rain['Jun-Sep'][60:90]))
rain_fall_data = list(rain['Jun-Sep'][rain_s:rain_e])
rain_data = []
for i in range(len(rain_fall_data)):
  temp = (rain_fall_data[i]/lpa)*100
  rain_data.append(temp)

In [24]:
# Extracing Data
def get_feature(data):
  new_data = []
  for i in range(len(data)):
    d = data.iloc[i]
    new_data.append(list(d[1:]))
  return np.array(new_data)

# Stacking Data of All Months
combine_slp, combine_sst, combine_uwnd = [], [], []
for i in range(12):
  month_path_slp, month_path_sst, month_path_uwnd= slp_months[i], sst_months[i], uwnd_months[i] 
  month_data_slp ,month_data_sst, month_data_uwnd= pd.read_csv(month_path_slp), pd.read_csv(month_path_sst), pd.read_csv(month_path_uwnd)
 
  feature_slp, feature_sst, feature_uwnd = get_feature(month_data_slp), get_feature(month_data_sst), get_feature(month_data_uwnd)
  feature_slp, feature_sst, feature_uwnd = feature_slp[slp_s:slp_e], feature_sst[sst_s:sst_e], feature_uwnd[uwnd_s:uwnd_e]
  for i in feature_slp:
    combine_slp.append(i)
  for i in feature_sst:
    combine_sst.append(i)
  for i in feature_uwnd:
    combine_uwnd.append(i)
x_slp = np.array(combine_slp)
x_sst = np.array(combine_sst)
x_uwnd = np.array(combine_uwnd)
x = np.concatenate((x_slp, x_sst, x_uwnd), axis=1)
x.shape, x_slp.shape, x_sst.shape, x_uwnd.shape

((636, 713), (636, 324), (636, 65), (636, 324))

In [25]:
model_1 = get_model([713, 47])
model_1.summary()

[713, 47]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 713)]             0         
                                                                 
 dense (Dense)               (None, 713)               509082    
                                                                 
 re_lu (ReLU)                (None, 713)               0         
                                                                 
 dense_1 (Dense)             (None, 47)                33558     
                                                                 
 dense_2 (Dense)             (None, 713)               34224     
                                                                 
Total params: 576,864
Trainable params: 576,864
Non-trainable params: 0
_________________________________________________________________


In [26]:
# fit the autoencoder model to reconstruct input
history = model_1.fit(x, x, epochs=800, batch_size=16, verbose=2, validation_data=(x,x))
# plot loss

Epoch 1/800
40/40 - 1s - loss: 15364.2559 - val_loss: 10932.5791 - 545ms/epoch - 14ms/step
Epoch 2/800
40/40 - 0s - loss: 9860.0684 - val_loss: 8502.5889 - 152ms/epoch - 4ms/step
Epoch 3/800
40/40 - 0s - loss: 7796.7979 - val_loss: 6874.5220 - 144ms/epoch - 4ms/step
Epoch 4/800
40/40 - 0s - loss: 6470.7163 - val_loss: 5860.9150 - 148ms/epoch - 4ms/step
Epoch 5/800
40/40 - 0s - loss: 5693.2383 - val_loss: 5253.6318 - 144ms/epoch - 4ms/step
Epoch 6/800
40/40 - 0s - loss: 5117.6797 - val_loss: 4763.3164 - 158ms/epoch - 4ms/step
Epoch 7/800
40/40 - 0s - loss: 4696.4902 - val_loss: 4402.1182 - 149ms/epoch - 4ms/step
Epoch 8/800
40/40 - 0s - loss: 4326.0024 - val_loss: 4054.2817 - 161ms/epoch - 4ms/step
Epoch 9/800
40/40 - 0s - loss: 4045.8938 - val_loss: 3820.9929 - 152ms/epoch - 4ms/step
Epoch 10/800
40/40 - 0s - loss: 3825.4761 - val_loss: 3622.9563 - 138ms/epoch - 3ms/step
Epoch 11/800
40/40 - 0s - loss: 3677.1855 - val_loss: 3494.5908 - 148ms/epoch - 4ms/step
Epoch 12/800
40/40 - 0s - l

In [27]:
model_2 = get_model([713, 49, 29])
model_2.summary()

[713, 49, 29]
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 713)]             0         
                                                                 
 dense_3 (Dense)             (None, 713)               509082    
                                                                 
 re_lu_1 (ReLU)              (None, 713)               0         
                                                                 
 dense_4 (Dense)             (None, 49)                34986     
                                                                 
 re_lu_2 (ReLU)              (None, 49)                0         
                                                                 
 dense_5 (Dense)             (None, 29)                1450      
                                                                 
 dense_6 (Dense)             (None, 49)      

In [28]:
weight_transfer_layer_model_2 = [1,2,3,8]
weight_transfer_from_layer_model_1 = [1,2,3,4]

for i in range(len(weight_transfer_layer_model_2)):
  model_2_index = weight_transfer_layer_model_2[i]
  model_1_index = weight_transfer_from_layer_model_1[i]
  model_2.layers[model_2_index] = model_1.layers[model_1_index]
  model_2.layers[model_2_index].trainable = False

In [29]:
model_2.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 713)]             0         
                                                                 
 dense_3 (Dense)             (None, 713)               509082    
                                                                 
 re_lu_1 (ReLU)              (None, 713)               0         
                                                                 
 dense_4 (Dense)             (None, 49)                34986     
                                                                 
 re_lu_2 (ReLU)              (None, 49)                0         
                                                                 
 dense_5 (Dense)             (None, 29)                1450      
                                                                 
 dense_6 (Dense)             (None, 49)                1470

In [30]:
# fit the autoencoder model to reconstruct input
history = model_2.fit(x, x, epochs=800, batch_size=16, verbose=2, validation_data=(x,x))
# plot loss

Epoch 1/800
40/40 - 0s - loss: 16775.9512 - val_loss: 13540.3037 - 461ms/epoch - 12ms/step
Epoch 2/800
40/40 - 0s - loss: 12291.8623 - val_loss: 11173.2568 - 144ms/epoch - 4ms/step
Epoch 3/800
40/40 - 0s - loss: 10746.9609 - val_loss: 10073.1338 - 151ms/epoch - 4ms/step
Epoch 4/800
40/40 - 0s - loss: 9808.9590 - val_loss: 9214.3027 - 160ms/epoch - 4ms/step
Epoch 5/800
40/40 - 0s - loss: 8930.8105 - val_loss: 8303.3320 - 145ms/epoch - 4ms/step
Epoch 6/800
40/40 - 0s - loss: 8089.3311 - val_loss: 7624.2720 - 152ms/epoch - 4ms/step
Epoch 7/800
40/40 - 0s - loss: 7507.6362 - val_loss: 7157.4585 - 148ms/epoch - 4ms/step
Epoch 8/800
40/40 - 0s - loss: 7061.2563 - val_loss: 6784.2690 - 142ms/epoch - 4ms/step
Epoch 9/800
40/40 - 0s - loss: 6762.8481 - val_loss: 6532.6709 - 145ms/epoch - 4ms/step
Epoch 10/800
40/40 - 0s - loss: 6495.9556 - val_loss: 6229.9512 - 140ms/epoch - 4ms/step
Epoch 11/800
40/40 - 0s - loss: 6232.0938 - val_loss: 6035.7129 - 142ms/epoch - 4ms/step
Epoch 12/800
40/40 - 0s

In [31]:
model_3 = get_model([713, 49, 29, 9])
model_3.summary()

[713, 49, 29, 9]
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 713)]             0         
                                                                 
 dense_8 (Dense)             (None, 713)               509082    
                                                                 
 re_lu_4 (ReLU)              (None, 713)               0         
                                                                 
 dense_9 (Dense)             (None, 49)                34986     
                                                                 
 re_lu_5 (ReLU)              (None, 49)                0         
                                                                 
 dense_10 (Dense)            (None, 29)                1450      
                                                                 
 re_lu_6 (ReLU)              (None, 29)   

In [32]:
weight_transfer_layer_model_3 = [1, 2, 3, 4, 5, 10, 11, 12]
weight_transfer_from_layer_model_2 = [1, 2, 3, 4, 5, 6, 7, 8]

for i in range(len(weight_transfer_layer_model_3)):
  model_3_index = weight_transfer_layer_model_3[i]
  model_2_index = weight_transfer_from_layer_model_2[i]
  model_3.layers[model_3_index] = model_2.layers[model_2_index]
  model_3.layers[model_3_index].trainable = False

In [33]:
model_3.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 713)]             0         
                                                                 
 dense_8 (Dense)             (None, 713)               509082    
                                                                 
 re_lu_4 (ReLU)              (None, 713)               0         
                                                                 
 dense_9 (Dense)             (None, 49)                34986     
                                                                 
 re_lu_5 (ReLU)              (None, 49)                0         
                                                                 
 dense_10 (Dense)            (None, 29)                1450      
                                                                 
 re_lu_6 (ReLU)              (None, 29)                0   

In [34]:
# fit the autoencoder model to reconstruct input
history = model_3.fit(x, x, epochs=1000, batch_size=16, verbose=2, validation_data=(x,x))
# plot loss

Epoch 1/1000
40/40 - 1s - loss: 16885.0352 - val_loss: 14248.6553 - 518ms/epoch - 13ms/step
Epoch 2/1000
40/40 - 0s - loss: 13144.2979 - val_loss: 12240.7656 - 160ms/epoch - 4ms/step
Epoch 3/1000
40/40 - 0s - loss: 11942.4248 - val_loss: 11545.7666 - 149ms/epoch - 4ms/step
Epoch 4/1000
40/40 - 0s - loss: 11362.9941 - val_loss: 10899.4346 - 159ms/epoch - 4ms/step
Epoch 5/1000
40/40 - 0s - loss: 10754.7441 - val_loss: 10399.0859 - 175ms/epoch - 4ms/step
Epoch 6/1000
40/40 - 0s - loss: 10302.4902 - val_loss: 9954.8291 - 146ms/epoch - 4ms/step
Epoch 7/1000
40/40 - 0s - loss: 9931.6299 - val_loss: 9755.7520 - 155ms/epoch - 4ms/step
Epoch 8/1000
40/40 - 0s - loss: 9698.0869 - val_loss: 9396.6777 - 151ms/epoch - 4ms/step
Epoch 9/1000
40/40 - 0s - loss: 9428.5264 - val_loss: 9200.4404 - 147ms/epoch - 4ms/step
Epoch 10/1000
40/40 - 0s - loss: 9218.7520 - val_loss: 9008.5352 - 146ms/epoch - 4ms/step
Epoch 11/1000
40/40 - 0s - loss: 9061.7334 - val_loss: 8854.6807 - 156ms/epoch - 4ms/step
Epoch 1

In [36]:
model_final = get_model([713, 49, 29, 9])
for i in range(13):
    model_final.layers[i] = model_3.layers[i]
model_final.summary()

[713, 49, 29, 9]
Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 713)]             0         
                                                                 
 dense_22 (Dense)            (None, 713)               509082    
                                                                 
 re_lu_14 (ReLU)             (None, 713)               0         
                                                                 
 dense_23 (Dense)            (None, 49)                34986     
                                                                 
 re_lu_15 (ReLU)             (None, 49)                0         
                                                                 
 dense_24 (Dense)            (None, 29)                1450      
                                                                 
 re_lu_16 (ReLU)             (None, 29)   

In [37]:
history = model_final.fit(x, x, epochs=1000, batch_size=16, verbose=2, validation_data=(x,x))

Epoch 1/1000
40/40 - 1s - loss: 17451.1875 - val_loss: 14848.6992 - 532ms/epoch - 13ms/step
Epoch 2/1000
40/40 - 0s - loss: 13413.9668 - val_loss: 12150.0234 - 154ms/epoch - 4ms/step
Epoch 3/1000
40/40 - 0s - loss: 11839.3652 - val_loss: 11357.6289 - 146ms/epoch - 4ms/step
Epoch 4/1000
40/40 - 0s - loss: 11195.4570 - val_loss: 10713.2646 - 162ms/epoch - 4ms/step
Epoch 5/1000
40/40 - 0s - loss: 10651.4453 - val_loss: 10362.7246 - 154ms/epoch - 4ms/step
Epoch 6/1000
40/40 - 0s - loss: 10353.5557 - val_loss: 10131.0557 - 163ms/epoch - 4ms/step
Epoch 7/1000
40/40 - 0s - loss: 10113.5586 - val_loss: 9963.0693 - 149ms/epoch - 4ms/step
Epoch 8/1000
40/40 - 0s - loss: 9946.6201 - val_loss: 9736.9629 - 149ms/epoch - 4ms/step
Epoch 9/1000
40/40 - 0s - loss: 9780.4570 - val_loss: 9629.5293 - 160ms/epoch - 4ms/step
Epoch 10/1000
40/40 - 0s - loss: 9657.2090 - val_loss: 9465.0186 - 163ms/epoch - 4ms/step
Epoch 11/1000
40/40 - 0s - loss: 9510.0312 - val_loss: 9317.9551 - 146ms/epoch - 4ms/step
Epoch

In [38]:
model_final1 = Model(inputs=model_final.input, outputs=model_final.layers[1].output)
model_final1.trainable = False

model_final2 = Model(inputs=model_final.input, outputs=model_final.layers[3].output)
model_final2.trainable = False

model_final3 = Model(inputs=model_final.input, outputs=model_final.layers[5].output)
model_final3.trainable = False

model_final4 = Model(inputs=model_final.input, outputs=model_final.layers[7].output)
model_final4.trainable = False

models = [model_final1, model_final2, model_final3, model_final4]
# models = [model_final1, model_final4]

In [39]:
# To extract only the relavant infromation from data frame for autoencoder
def get_feature(data):
    new_data = []
    for i in range(len(data)):
      d = data.iloc[i]
      new_data.append(list(d[1:]))
    # print(new_data)
    return np.array(new_data)

# to combine all the features together
def predictor(month,model_final):
    month_path_slp, month_path_sst, month_path_uwnd= slp_months[month], sst_months[month], uwnd_months[month] 
    month_data_slp ,month_data_sst, month_data_uwnd= pd.read_csv(month_path_slp), pd.read_csv(month_path_sst), pd.read_csv(month_path_uwnd)
  
    feature_slp, feature_sst, feature_uwnd = get_feature(month_data_slp), get_feature(month_data_sst), get_feature(month_data_uwnd)
    feature_slp, feature_sst, feature_uwnd = feature_slp[slp_s:slp_e], feature_sst[sst_s:sst_e], feature_uwnd[uwnd_s:uwnd_e]
    
    feature = np.concatenate((feature_slp, feature_sst, feature_uwnd), axis=1)

    pred = np.array(feature)
    pred.shape
    pred_m = model_final(pred)
    pred_f = pred_m.numpy()
    return pred_f

In [ ]:
#This gave 0.76 for month feb
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.svm import SVR

tree = DecisionTreeRegressor(max_depth=4)
from sklearn.ensemble import RandomForestRegressor
cor_all = []
reg_f = SVR(kernel = 'rbf')
for i in range(1,6):
  for j in range(40):
    reg = BaggingRegressor( n_estimators=i, random_state=j).fit(pred[:53], rain_fall_data[:53])
    regr = RandomForestRegressor(max_depth=i, random_state=j)
    reg_f.fit(pred[:53], rain_fall_data[:53])
    preds = reg_f.predict(pred[53:67])
    corr, _ = pearsonr(rain_fall_data[53:67],preds)
    cor_all.append(corr)
    print("depth ",i,"    ",corr)
print(min(cor_all),"   ",max(cor_all))

depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      

In [46]:
from sklearn.svm import SVR
from scipy.stats import pearsonr
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


#w_start = window start, w_end = window end  
def get_top_pred(top,pred_info,w_start,w_end):
  pred_f = pred_info
  cor_list = []
  for i in range(len(pred_f[0])):
    score = []
    
    for j in range(w_start,w_end):
      score.append(pred_f[j][i])
    print(len(rain_fall_data[w_start:w_end]), len(score))
    corr, _ = pearsonr(rain_fall_data[w_start:w_end],score)
    cor_list.append(corr)
    # print(corr)
  list1=list(enumerate(cor_list)) #I have taken cosine rank[0] beacuse it is inside the list
  list2=sorted(list1, key=lambda x: x[1],reverse=True)
  top_feature_index = []
  #here i am getting the top features index
  for i in range(top):
    index = list2[i][0]
    top_feature_index.append(index)
  # Here i am getting all the top features
  predictor = []
  for i in range(len(pred_f)):
    temp = []
    for j in top_feature_index:
      # print(i,"  ",j)
      feature = pred_f[i][j]
      temp.append(feature)
    predictor.append(temp)
  # print(list2)
  return predictor
  # print(list1)


# np.concatenate((a, b.T), axis=1)
def get_predictors(months,top,models,w_start,w_end):
  y_pred = predictor(months[0],models[0])
  # print(y_pred.shape)
  pred = get_top_pred(top, y_pred, w_start,w_end)
  # print(len(pred[0]))
  pred = np.array(pred)
  for mod in models[1:]:
    y_pred = predictor(months[0],mod)
    b = get_top_pred(top, y_pred,w_start,w_end)
    pred = np.concatenate((pred,b), axis=1)
  if(len(months)>1):
    for i in months[1:]:
      for mod in models[0:]:
        y_pred = predictor(i,mod)
        b = get_top_pred(top, y_pred, w_start,w_end)
        pred = np.concatenate((pred,b), axis=1)
  return pred

def window_solution(months,top):
  reg = SVR(kernel = 'rbf',C=1.0,epsilon=0.45)
  # reg = make_pipeline(StandardScaler(), SVR(kernel = 'rbf',C=1.0, epsilon=0.2))
  # months = [0]
  k=0
  cor_all = []
  for window in range(10,53):
    k+=1
    score = []
    pred = get_predictors(months,top,models,53-window,53)
    for i in range(14):
      start = 53-window
      end = 53+1
      # pred = get_predictors(months,2,models,start,end)
      reg.fit(pred[53-window:53+i], rain_fall_data[53-window:53+i])
      score.append(reg.predict([pred[53+i]])[0])
    # print(score)
    corr, _ = pearsonr(rain_fall_data[53:67],score)
    print("Window size = ",window,"   plcc",corr)
    cor_all.append(corr)
  return min(cor_all),max(cor_all)
#This is by taking correlation after every iteraion of each predict

In [47]:
months_comb = [[4]]
top = 9
models = [model_final3 ]
for month in months_comb:
  for i in range(1,top):
    left, right = window_solution(month,i)
    print(" Month combination = ",month,"    top = ",i,"   min and max",left,"   ",right)

9 10


ValueError: x and y must have the same length.